# Advanced Lane Detection 


The following steps were performed to obtain the lane detections

1) For the given set of chessboard images, the caliberation matrix and the distortion coefficients were calculated using the call_undistort. Using these images we get the object points and image points. For our case, the chessboard size is chosen to be 9X6. An example of undistorted image is saved in output_images folder as undistorted_chessboard.jpg A sample output is saved at **./CarND-Advanced-Lane-Lines/output_images/undistorted_chessboard.jpg**


# PIPELINE(Single Images)


### Distortion Corrected Image 
From the above extracted caliberation matrix and distortion coefficients, a test image was corrected for distortion using the function call_undistort. A sample output is saved at **./CarND-Advanced-Lane-Lines/output_images/undistorted_testImage.jpg**.

### Binary Image Extraction 

The binary thresholded image is obtained from the undistorted image via binary_image function saved at **./CarND-Advanced-Lane-Lines/output_images/binary_testImage.jpg**. Here we initially use the HLS color transform, LUV, and LAB color transforms. We use the saturation channel. And we binary threshold it between 



    
    ######## Function for retreiving a binary thresholded image ##########
    def binary_image(img, s_thresh=(170, 255), sx_thresh=(20, 100),l_thresh = (225,255),b_thresh = (155,200)):
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)   ###### Converting RGB to HLS color format
        s_channel = hls[:,:,2]   ###### Consdiering only the Saturation channel, as opposed to Lightness

    # Grayscale image
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    luv = cv2.cvtColor(img,cv2.COLOR_RGB2Luv)
    lab = cv2.cvtColor(img,cv2.COLOR_RGB2Lab) 
    
    
    l_channel = luv[:,:,0]
    b_channel = lab[:,:,2]
    
    # Sobel x Finding gradients along X and Y directions. 
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1) # Take the derivative in y
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    abs_sobely = np.absolute(sobely) # Absolute x derivative to accentuate lines away from horizontal
    abs_sobelxy = np.sqrt(abs_sobelx**2 + abs_sobely**2)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx)) # Converting into unsigned interger format
    sxbinary = np.zeros_like(scaled_sobel)
    # Applying appropriate thresholding make all pixels between the low and high threshold values as 1 
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1 
    
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    # Combine the two binary thresholds
    #### Combining binary images from sobel_x filter and s_channel images 
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(b_binary == 1) | (l_binary == 1) | (s_binary == 1) | (sxbinary == 1)] =1
    return combined_binary

For the saturation channel we use low and high threshold as 170, 255 respectively between which all are converted to 1 else 0. The other one is by considering the sobel filter gradient along the x direction. And the thresholds for this is chosen to be 20, 100. For L channel from LUV transform, we use thresholds l_thresh = (225,255) and for B channel from LAB color transform we use between 155 and 200. LUV and LAB color transforms helps in detection of yellow lanes and works well under shadowy images

THe result of these two binary images are combined in such a way that if either of the pixels are 1, the result is a 1 else 0.


### Perspective Transform

From the binary thresholded image, the perspective transform is performed by function return_warped_image saved at **./CarND-Advanced-Lane-Lines/output_images/perspective_testImage.jpg**, in which src and dst points were hardcoded as follows. 



    
    
    def return_warped_image(img,x_adj=0):
    img_height = img.shape[0]
    img_width = img.shape[1]
    imshape = (img_height, img_width)
    xcenter=imshape[1]/2+x_adj
    xfd=50
    yf=450
    offset=100
    ######## Calculating source and destination points to be mapped ##########
    ###### Based on the test images an offset of 100 is provided ###########
    src = np.float32(
        [(offset,imshape[0]),
         (xcenter-xfd, yf), 
         (xcenter+xfd,yf), 
         (imshape[1]-offset,imshape[0])])
    
    dst = np.float32(
        [(offset,imshape[1]),
         (offset,0),
         (imshape[0]-offset, 0),
        (imshape[0]-offset,imshape[1])])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img, M, imshape, flags=cv2.INTER_LINEAR)
        
    return warped, M , Minv
    
This function uses the perspective transform and warpperspective methods to obtain the perspective transform.

### Fitting polynomial 
Sliding window is not used to detect the lane. However the left and right lanes are indicated by red and blue colors respectively. A fitted polynomial is shown in yellow color saved at **./CarND-Advanced-Lane-Lines/output_images/fitted_polynomial_testImage.jpg.**
The lower part of the image is considered for histogram. Every column of the lower part of the image is summed up and its frequency distribution is plotted. Based on the two highest frequencies, a 2nd order polynomial is fit at those two pixel positions(along x axis) is plotted. 


    # HYPERPARAMETERS
    def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        #(win_xleft_high,win_y_high),(0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),
        #(win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        #pass # Remove this when you add your function
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img




    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fitx,right_fitx,ploty

### Radius of Curvature and offset

The radius of curvature is calculated in the measure_curvature_real function. And the result is printed out in Advance_Lane_Detection.ipynb.

### Final Result visual Output
For the video we see the distance between the left and right polynomials fit, if the distance between the fit is less than 300 or greater than 600 then we use the average of past 4 best frames. The distance between the fitted lines is a sanity check so that the distance won't go out of the standard lane.
The final edge detection is plotted onto the original image using the plot_detected_orig_image function saved at  **./CarND-Advanced-Lane-Lines/output_images/FinalResult_withdetection.jpg**

### Pipeline video. 
A complete_pipeline function is implemented to run all the above steps on the video using the moviePy.editor module from python 



    ######## Building pipeline ########
    global left_fitxlist
    global right_fitxlist
    left_fitxlist = []
    right_fitxlist = []
    import pdb
    def complete_pipeline(img):

    global prev_leftx
    global prev_rightx
    global prev_ploty
    

    
    img_size = (img.shape[1], img.shape[0])
    #img = cv2.imread(testImagepath[0])
    #undist = cal_undistort(img, objpoints, imgpoints)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    binaryImage = binary_image(undist)
    perspectiveTransformImage,M,Minv = return_warped_image(binaryImage)
    fitPolynomial,left_fitx,right_fitx,ploty = fit_polynomial(perspectiveTransformImage)
    

    
    
    #left_curvature= round(measure_curvature_real(left_fitx,perspectiveTransformImage.shape[0]),2)
    #right_curvature  = round(measure_curvature_real(right_fitx,perspectiveTransformImage.shape[0]),2)
    
    left_curvature= round(measure_curvature_realAlt(ploty,left_fitx),2)
    right_curvature  = round(measure_curvature_realAlt(ploty,right_fitx),2)
    print(left_fitx)
    
    curv = (left_curvature+right_curvature)/2
    curvature = "left Rad: {} m , Right Rad: {} m".format(left_curvature,right_curvature)
    print(curvature)
    offset = measure_offset(left_fitx,right_fitx,perspectiveTransformImage.shape[0],img_size[0])
    offset = offset - 1
    off = "Offset: %.2f m" % offset
    distance = np.abs(left_fitx-right_fitx)
    if (min(distance)>300 and max(distance)<600):
        result= plot_detected_orig_image(off,curvature,img,undist,perspectiveTransformImage,Minv,left_fitx,right_fitx,ploty)
        #prev_leftx = left_fitx
        #prev_rightx = right_fitx
        left_fitxlist.append(left_fitx)
        right_fitxlist.append(right_fitx)
        try:
            prev_leftx = (left_fitxlist[-1]+left_fitxlist[-2]+left_fitxlist[-3]+left_fitxlist[-4])/4
            prev_rightx = (right_fitxlist[-1]+right_fitxlist[-2]+right_fitxlist[-3]+right_fitxlist[-4])/4
            prev_ploty = ploty
        except:
            prev_leftx = left_fitxlist[-1]
            prev_rightx = right_fitxlist[-1]
            prev_ploty = ploty
            
        #avg_leftx = 
    else:
        #pdb.set_trace()
        result= plot_detected_orig_image(off,curvature,img,undist,perspectiveTransformImage,Minv,prev_leftx,prev_rightx,prev_ploty)

      return result
    
The final project video is saved at **./CarND-Advanced-Lane-Lines/output_images/project_video_alternate.mp4**

###### Future Work
The project works better than the cannhy edge detection. However, it likely fails on the challenge_video.mp4. A better way to work on challenge video is to play around with the threshold values for obtaining binary images. And this video won't fair better on videos with lanes of variable width. Hard coding in that scenario wouldn't work. A better way would be to get adaptable width. 